# Import

In [26]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Exploration

Chargez le dataset IMDB via pandas. (https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews.) 
- Ce dataset tres connu rassemble des critiques de films de spectateurs
- L'objectif est d'entrainer un modèle de NLP à détecter si la critique est positive ou negative

In [27]:
data = pd.read_csv('IMDB_Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


Explorez le jeu de données en répondant aux questions suivantes :
- Combien y-t-il d'elements ?
- Quel est le langage ? A quoi ressemble les commentaires ? 
- Quelles longueur font les commentaires (moyenne ou mieux, visualiser la répartition du nombre de mots) ?
- Y-a-t-il suffisamment d'élements pour la tache ?
- est-il équilibré ? combien de critiques positives ou negatives ?

# Séparation

Il nous faut maintenant extraire et encoder les labels. Vous pouvez utiliser LabelEncoder de scikit learn pour le faire.

In [28]:
le = preprocessing.LabelEncoder()
le.fit(data['sentiment'])
y = le.transform(data['sentiment'])

- Puis nous séparons en train et validation

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(data["review"], y, test_size=0.2, random_state=42)

# Nettoyage 

Utilisons un bag of word pour encoder les textes. La classe correspondante dans scikit-learn est CountVectoriser. 
Son paramètre max_feature permet de limiter le nombre de colonnes créées si besoin (RAM, temps de calcul, mots peu importants, ...).

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

X_train_bow = CountVectorizer().fit_transform(X_train)
f"On obtient {X_train_bow.shape[1]} colonnes, chacune encodant le nombre d'occurence d'un mot dans les commentaires"

"On obtient 93003 colonnes, chacune encodant le nombre d'occurence d'un mot dans les commentaires"

# Création du model avec Keras

In [31]:
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow import keras

Ameliorez le réseau suivant :

    - je vous impose la syntaxe suivante pour construire le réseau plutôt que Sequential, pour une bonne raison ! Une intuition ?

In [32]:

input_layer = Input([1000])
out_layer = Dense(1)(input_layer)

model = keras.Model(inputs=input_layer, outputs=out_layer)
model.build([1000])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense_1 (Dense)             (None, 1)                 1001      
                                                                 
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


## Compilation

- Creez un optimizer et compilez le modele

In [37]:
X_val_bow, Y_train_bow = train_test_split(X_train_bow, y, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [40000, 50000]

In [34]:
X_train_bow.shape

(40000, 93003)

In [33]:
dropout_rate = 0.2
input_shape = X_val_bow.shape[1]

input_layer = Input([input_shape])
out_layer = Dropout(dropout_rate)(input_layer)
out_layer = Dense(units=10, activation='relu')(out_layer)
out_layer = Dropout(dropout_rate)(out_layer)
out_layer = Dense(units=1, activation='sigmoid')(out_layer)

SyntaxError: invalid syntax (2390182990.py, line 5)

In [18]:

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000)]            0         
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


# Entrainement

In [20]:
# Pour ajouter des comportements additionnels appelés lors de l'entrainement. Par exemple des logs, de l'early stopping, ou tout méthode custom
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=2)]
epochs = 1
batch_size = 1


history = model.fit(X_train_bow, 
                    y_train, 
                    epochs=epochs, 
                    validation_data=(X_val_bow, y_val), 
                    verbose=1, 
                    batch_size=batch_size, 
                    callbacks=callbacks)

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

Ameliorez les résultats précédents :
- en modifiant ou ajoutant des couches au réseau
- en modifiant les parametres (learning_rate, optimizer, loss, ...)
- en augmentant la taille de batch et le nombre d'époques

# Analyse des résultats

Appliquez un seuil de décision à la sortie de votre réseau
- sortie = probabilité
- le seuil de décision revient à dire à partir de quelle proba vous considérez que le sentiment est positif.
- par défaut, 0.5, mais on peut parfois faire mieux

In [ ]:
y_val_pred = model.predict(X_val_bow)

Afficher la matrice de confusion pour analyser les résultats

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

cf_matrix =None
print(cf_matrix)

0 : sentiment negatif
1 : sentiment positif

Sur quelle classe le modele performe-t-il le mieux ?

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, 
            fmt='.2%', cmap='Blues')

Finalement, déterminer s'il y a des longueurs de critiques pour lesquelles on prédit moins bien